# Predictive maintenance of Lathe machine

#### Importing important libraries and modules

In [1]:
import pandas as pd
import numpy as np
import torch 
import tensorflow as tf
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal

#### Importing Vibration Data and converting it to proper Time-series format.

In [4]:
dataframes = []  # To store the imported data from each file

for i in range(1, 61):
    file = f"Data/{i}.xlsx"
    df = pd.read_excel(file)  # Use pd.read_excel() for Excel files
    df = df.dropna(axis='columns', how='all')  
    df = df.dropna(axis='rows', how='all') 
    df.columns = ['Time', 'X', 'Y', 'Z']
    df = df.iloc[1:]  # Exclude the original header row from the data
    df['Time'] = pd.to_datetime(df['Time'], unit='s').dt.time  # Convert 'Time' column to datetime
    dataframes.append(df)
    print(i) #too keep an eye on progress


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [3]:
exp = pd.read_excel("Data/Experiment Summary.xlsx")

In [3]:
merged_df
merged_df = pd.concat(dataframes, ignore_index=True)

,Time,X,Y,Z
0,00:00:00,-0.961714,-2.247089,-4.40107
1,00:00:00.000976,-0.394591,-2.298639,-1.552597
2,00:00:00.001953,-5.44346,0.849591,5.49498
3,00:00:00.002929,-1.46255,0.77963,1.958311
4,00:00:00.003906,1.148426,1.03738,-1.342826
...,...,...,...,...
1952219,00:00:18.495000,-3.362781,-3.635256,-1.346506
1952220,00:00:18.496000,-0.018964,1.722258,5.715792
1952221,00:00:18.497000,0.809625,5.128238,-0.1026
1952222,00:00:18.498000,-1.68719,-1.746318,-1.291303


In [ ]:
for i in dataframes:
    d['X'] = pd.to_numeric(d['X'], errors='coerce')
    d['Y'] = pd.to_numeric(d['Y'], errors='coerce')
    d['Z'] = pd.to_numeric(d['Z'], errors='coerce')
    

#### Feature Extraction

In [ ]:
for d in dataframes:
    d['Magnitude'] = np.sqrt(d['X']**2 + d['Y']**2 + d['Z']**2)

In [ ]:
#view imported data.

for i in range(60):
    print(i)
    print(dataframes[i].head(5), dataframes[i].shape)

In [ ]:
# Time_domain plot


def time_domain(num):
    df = dataframes[num]

    # Create a line plot using Plotly
    fig = px.line(df, x='Time', y=['Magnitude'], title='Vibration Sensor Data', labels={'value':'acceleration'})
    
    # Display the plot
    fig.update_layout(
        height=600,
        showlegend=True,
        paper_bgcolor= 'darkgrey')
    fig.show()
    

    
# Calculating PSD

def calculate_psd(dataframe, fs=1000):
    time = dataframe['Time']
    x = dataframe['Magnitude']

    # Convert time values to seconds
    time_seconds = [(t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6) for t in time]

    # Apply Hanning window function
    window = np.hanning(len(time_seconds))
    x_windowed = x * window
 

    # Calculate PSD using periodogram
    f, psd_x = signal.periodogram(x_windowed, fs)


    return f, psd_x



In [ ]:
#Plotting PSD


def freq_domain(frequencies, psd_x):
    # Convert complex PSD values to magnitude
    psd_x_mag = np.abs(psd_x)

    # Create line plots for X, Y, and Z axes
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=frequencies, y=psd_x_mag, mode='lines', name='Magnitude'))

    fig.update_layout(
        title='Power Spectral Density',
        xaxis=dict(title='Frequency'),
        yaxis=dict(title='acceleration', type='log', range=[np.log10(1), np.log10(max(psd_x_mag))]),
        showlegend=True,
        paper_bgcolor= 'darkgrey',
        height=600


    )

    fig.show()




In [ ]:

# Calculating PSD

from scipy import signal
from scipy.signal import welch

def calculate_psd(dataframe, fs=1000, w=1):
    time = dataframe['Time']
    x = dataframe['X']
    y = dataframe['Y']
    z = dataframe['Z']

    # Convert time values to seconds
    time_seconds = [(t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6) for t in time]

    # Apply Hanning window function
    window = np.hanning(len(time_seconds))
    x_windowed = x * window
    y_windowed = y * window
    z_windowed = z * window

    # Calculate PSD using periodogram
    f, psd_x = welch(x_windowed, fs, nperseg=len(time_seconds)/w)
    _, psd_y = welch(y_windowed, fs, nperseg=len(time_seconds)/w)
    _, psd_z = welch(z_windowed, fs, nperseg=len(time_seconds)/w)

    return f, psd_x, psd_y, psd_z



In [ ]:
# Time_domain plot


def time_domain(num):
    df = dataframes[num]

    # Create a line plot using Plotly
    fig = px.line(df, x='Time', y=['X', 'Y', 'Z'], title='Vibration Sensor Data', labels={'value':'acceleration'})
    
    # Display the plot
    fig.update_layout(
        height=600,
        showlegend=True,
        paper_bgcolor= 'darkgrey')
    fig.show()
    


def freq_domain(frequencies, psd_x, psd_y, psd_z):
    # Convert complex PSD values to magnitude
    psd_x_mag = np.abs(psd_x)
    psd_y_mag = np.abs(psd_y)
    psd_z_mag = np.abs(psd_z)

    # Create line plots for X, Y, and Z axes
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=frequencies, y=psd_x_mag, mode='lines', name='X'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_y_mag, mode='lines', name='Y'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_z_mag, mode='lines', name='Z'))

    fig.update_layout(
        title='Power Spectral Density',
        xaxis=dict(title='Frequency'),
        yaxis=dict(title='acceleration'),
        showlegend=True,
        paper_bgcolor= 'darkgrey',
        height=600


    )

    fig.show()

### Time Domain, Frequency Domain and time vs frequency charts:

##### RPM: 190

In [ ]:
from scipy.signal import find_peaks

# Calling the function with the DataFrames
for j in range(5):
    for i in range(j, 60, 20):
        print(exp.iloc[i])
        new = time_domain(i) 
        experiment, x, y, z = calculate_psd(dataframes[i], fs=500, w=32)
        plo = freq_domain(experiment, x, y, z)
        
        print()
    